In [1]:
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
#from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive
)
import math
from datetime import datetime

In [2]:
micro_pool = pd.read_csv('micro_pool_socio_bio2up.csv')
micro_pool['popdensityk']= micro_pool['popdensity']/1000

database = db.Database('micro', micro_pool)
# They are organized as panel data. The variable who identifies each individual.
#database.panel("who") # remember to sort data by individual

globals().update(database.variables)

In [3]:
# S and RND ascs are for panel data. Ignore for calibration
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_ANTI')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_ANTI')

ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0)
ASC_WALK_RND = ASC_WALK + ASC_WALK_S * bioDraws('ASC_WALK_RND', 'NORMAL_ANTI')

ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_BIKE_S = Beta('ASC_BIKET_S', 1, None, None, 0)
ASC_BIKE_RND = ASC_BIKE + ASC_BIKE_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')


B_carTIME = Beta('B_carTIME', 0, None, None, 0) #name, default value, lower bound, upper bound, estimate(Y0,N1)
B_transitTIME = Beta('B_transitTIME', 0, None, None, 0)
B_rhTIME = Beta('B_rhTIME', 0, None, None, 0)
B_walkTIME = Beta('B_walkTIME', 0, None, None, 0)
B_bikeTIME = Beta('B_bikeTIME', 0, None, None, 0)

B_scooterTIME = Beta('B_scooterTIME', 0, None, None, 0)
B_sctransitTIME = Beta('B_sctransitTIME', 0, None, None, 0)

B_access = Beta('B_access', 0, None, None, 0)
B_drop =Beta('B_drop', 0, None, None, 0)
B_waitav = Beta('B_waitav', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_PRCP1 = Beta('B_PRCP1', 0, None, None, 0)#Heavy rain
B_PRCP2 = Beta('B_PRCP2', 0, None, None, 0)#Light rain, ref: No rain

B_bikePRCP1 = Beta('B_bikePRCP1', 0, None, None, 0)#Heavy rain
B_bikePRCP2 = Beta('B_bikePRCP2', 0, None, None, 0)#Light rain, ref: No rain

B_scooterPRCP1 = Beta('B_scooterPRCP1', 0, None, None, 0)#Heavy rain
B_scooterPRCP2 = Beta('B_scooterPRCP2', 0, None, None, 0)#Light rain, ref: No rain


B_bikeBKLN2 = Beta('B_bikeBKLN2', 0, None, None, 0)#80% bikelane
B_bikeBKLN3 = Beta('B_bikeBKLN3', 0, None, None, 0)#half bikelane
B_bikeBKLN4 = Beta('B_bikeBKLN4', 0, None, None, 0)#no bikelane, ref: full bikelane

B_scooterBKLN2 = Beta('B_scooterBKLN2', 0, None, None, 0)#80% bikelane
B_scooterBKLN3 = Beta('B_scooterBKLN3', 0, None, None, 0)#half bikelane
B_scooterBKLN4 = Beta('B_scooterBKLN4', 0, None, None, 0)#no bikelane, ref: full bikelane

B_bikeBKLNn = Beta('B_bikeBKLNn', 0, None, None, 0)
B_scooterBKLNn = Beta('B_scooterBKLNn', 0, None, None, 0)

B_TEMPbike = Beta('B_TEMPbike', 0, None, None, 0)
B_TEMPsqbike = Beta('B_TEMPsqbike', 0, None, None, 0)

B_TEMPsc = Beta('B_TEMPsc', 0, None, None, 0)
B_TEMPsqsc = Beta('B_TEMPsqsc', 0, None, None, 0)

B_carHBW = Beta('B_carHBW', 0, None, None, 0)
B_carHBSHOP = Beta('B_carHBSHOP', 0, None, None, 0)
B_carHBSOC = Beta('B_carHBSOC', 0, None, None, 0)
B_carHBO = Beta('B_carHBO', 0, None, None, 0)
B_carNHB = Beta('B_carNHB', 0, None, None, 0)


B_carPURPHome = Beta('B_carPURPHome', 0, None, None, 0)#go back home
B_carPURPShop = Beta('B_carPURPShop', 0, None, None, 0)#shopping
B_carPURPErra = Beta('B_carPURPErra', 0, None, None, 0)#errand
B_carPURPScho = Beta('B_carPURPScho', 0, None, None, 0)#school/church
B_carPURPSoci = Beta('B_carPURPSoci', 0, None, None, 0)#social

B_carorigHome = Beta('B_carorigHome', 0, None, None, 0)
B_carorigShop = Beta('B_carorigShop', 0, None, None, 0)
B_carorigErra = Beta('B_carorigErra', 0, None, None, 0)
B_carorigScho = Beta('B_carorigScho', 0, None, None, 0)
B_carorigSoci = Beta('B_carorigSoci', 0, None, None, 0)

B_transitHBW = Beta('B_transitHBW', 0, None, None, 0)
B_transitHBSHOP = Beta('B_transitHBSHOP', 0, None, None, 0)
B_transitHBSOC = Beta('B_transitHBSOC', 0, None, None, 0)
B_transitHBO = Beta('B_transitHBO', 0, None, None, 0)
B_transitNHB = Beta('B_transitNHB', 0, None, None, 0)

B_transitPURPHome = Beta('B_transitPURPHome', 0, None, None, 0)#go back home
B_transitPURPShop = Beta('B_transitPURPShop', 0, None, None, 0)#shopping
B_transitPURPErra = Beta('B_transitPURPErra', 0, None, None, 0)#errand
B_transitPURPScho = Beta('B_transitPURPScho', 0, None, None, 0)#school/church
B_transitPURPSoci = Beta('B_transitPURPSoci', 0, None, None, 0)#social

B_transitorigHome = Beta('B_transitorigHome', 0, None, None, 0)
B_transitorigShop = Beta('B_transitorigShop', 0, None, None, 0)
B_transitorigErra = Beta('B_transitorigErra', 0, None, None, 0)
B_transitorigScho = Beta('B_transitorigScho', 0, None, None, 0)
B_transitorigSoci = Beta('B_transitorigSoci', 0, None, None, 0)

B_rhHBW = Beta('B_rhHBW', 0, None, None, 0)
B_rhHBSHOP = Beta('B_rhHBSHOP', 0, None, None, 0)
B_rhHBSOC = Beta('B_rhHBSOC', 0, None, None, 0)
B_rhHBO = Beta('B_rhHBO', 0, None, None, 0)
B_rhNHB = Beta('B_rhNHB', 0, None, None, 0)

B_rhPURPHome = Beta('B_rhPURPHome', 0, None, None, 0)#go back home
B_rhPURPShop = Beta('B_rhPURPShop', 0, None, None, 0)#shopping
B_rhPURPErra = Beta('B_rhPURPErra', 0, None, None, 0)#errand
B_rhPURPScho = Beta('B_rhPURPScho', 0, None, None, 0)#school/church
B_rhPURPSoci = Beta('B_rhPURPSoci', 0, None, None, 0)#social

B_rhorigHome = Beta('B_rhorigHome', 0, None, None, 0)
B_rhorigShop = Beta('B_rhorigShop', 0, None, None, 0)
B_rhorigErra = Beta('B_rhorigErra', 0, None, None, 0)
B_rhorigScho = Beta('B_rhorigScho', 0, None, None, 0)
B_rhorigSoci = Beta('B_rhorigSoci', 0, None, None, 0)

B_walkHBW = Beta('B_walkHBW', 0, None, None, 0)
B_walkHBSHOP = Beta('B_walkHBSHOP', 0, None, None, 0)
B_walkHBSOC = Beta('B_walkHBSOC', 0, None, None, 0)
B_walkHBO = Beta('B_walkHBO', 0, None, None, 0)
B_walkNHB = Beta('B_walkNHB', 0, None, None, 0)

B_walkPURPHome = Beta('B_walkPURPHome', 0, None, None, 0)#go back home
B_walkPURPShop = Beta('B_walkPURPShop', 0, None, None, 0)#shopping
B_walkPURPErra = Beta('B_walkPURPErra', 0, None, None, 0)#errand
B_walkPURPScho = Beta('B_walkPURPScho', 0, None, None, 0)#school/church
B_walkPURPSoci = Beta('B_walkPURPSoci', 0, None, None, 0)#social

B_walkorigHome = Beta('B_walkorigHome', 0, None, None, 0)
B_walkorigShop = Beta('B_walkorigShop', 0, None, None, 0)
B_walkorigErra = Beta('B_walkorigErra', 0, None, None, 0)
B_walkorigScho = Beta('B_walkorigScho', 0, None, None, 0)
B_walkorigSoci = Beta('B_walkorigSoci', 0, None, None, 0)

B_bikeHBW = Beta('B_bikeHBW', 0, None, None, 0)
B_bikeHBSHOP = Beta('B_bikeHBSHOP', 0, None, None, 0)
B_bikeHBSOC = Beta('B_bikeHBSOC', 0, None, None, 0)
B_bikeHBO = Beta('B_bikeHBO', 0, None, None, 0)
B_bikeNHB = Beta('B_bikeNHB', 0, None, None, 0)

B_bikePURPHome = Beta('B_bikePURPHome', 0, None, None, 0)#go back home
B_bikePURPShop = Beta('B_bikePURPShop', 0, None, None, 0)#shopping
B_bikePURPErra = Beta('B_bikePURPErra', 0, None, None, 0)#errand
B_bikePURPScho = Beta('B_bikePURPScho', 0, None, None, 0)#school/church
B_bikePURPSoci = Beta('B_bikePURPSoci', 0, None, None, 0)#social

B_bikeorigHome = Beta('B_bikeorigHome', 0, None, None, 0)
B_bikeorigShop = Beta('B_bikeorigShop', 0, None, None, 0)
B_bikeorigErra = Beta('B_bikeorigErra', 0, None, None, 0)
B_bikeorigScho = Beta('B_bikeorigScho', 0, None, None, 0)
B_bikeorigSoci = Beta('B_bikeorigSoci', 0, None, None, 0)

B_microHBW = Beta('B_microHBW', 0, None, None, 0)
B_microHBSHOP = Beta('B_microHBSHOP', 0, None, None, 0)
B_microHBSOC = Beta('B_microHBSOC', 0, None, None, 0)
B_microHBO = Beta('B_microHBO', 0, None, None, 0)
B_microNHB = Beta('B_microNHB', 0, None, None, 0)

B_microPURPHome = Beta('B_microPURPHome', 0, None, None, 0)#go back home
B_microPURPShop = Beta('B_microPURPShop', 0, None, None, 0)#shopping
B_microPURPErra = Beta('B_microPURPErra', 0, None, None, 0)#errand
B_microPURPScho = Beta('B_microPURPScho', 0, None, None, 0)#school/church
B_microPURPSoci = Beta('B_microPURPSoci', 0, None, None, 0)#social

B_microorigHome = Beta('B_microorigHome', 0, None, None, 0)
B_microorigShop = Beta('B_microorigShop', 0, None, None, 0)
B_microorigErra = Beta('B_microorigErra', 0, None, None, 0)
B_microorigScho = Beta('B_microorigScho', 0, None, None, 0)
B_microorigSoci = Beta('B_microorigSoci', 0, None, None, 0)


B_carage = Beta('B_carage', 0, None, None, 0)
B_transitage = Beta('B_transitage', 0, None, None, 0)
B_rhage = Beta('B_rhage', 0, None, None, 0)
B_walkage = Beta('B_walkage', 0, None, None, 0)
B_bikeage = Beta('B_bikeage', 0, None, None, 0)
B_microage = Beta('B_microage', 0, None, None, 0)

B_caragesq = Beta('B_caragesq', 0, None, None, 0)
B_transitagesq = Beta('B_transitagesq', 0, None, None, 0)
B_rhagesq = Beta('B_rhagesq', 0, None, None, 0)
B_walkagesq = Beta('B_walkagesq', 0, None, None, 0)
B_bikeagesq = Beta('B_bikeagesq', 0, None, None, 0)
B_microagesq = Beta('B_microagesq', 0, None, None, 0)

B_carFe = Beta('B_carFe', 0, None, None, 0)
B_transitFe = Beta('B_transitFe', 0, None, None, 0)
B_rhFe = Beta('B_rhFe', 0, None, None, 0)
B_walkFe = Beta('B_walkFe', 0, None, None, 0)
B_bikeFe = Beta('B_bikeFe', 0, None, None, 0)
B_scooterFe = Beta('B_scooterFe', 0, None, None, 0)
B_sharedbikeFe = Beta('B_sharedbikeFe', 0, None, None, 0)


B_carincomek = Beta('B_carincomek', 0, None, None, 0)
B_transitincomek = Beta('B_transitincomek', 0, None, None, 0)
B_rhincomek = Beta('B_rhincomek', 0, None, None, 0)
B_walkincomek = Beta('B_walkincomek', 0, None, None, 0)
B_bikeincomek = Beta('B_bikeincomek', 0, None, None, 0)
B_microincomek = Beta('B_microincomek', 0, None, None, 0)

B_carhedu = Beta('B_carhedu', 0, None, None, 0)
B_transithedu = Beta('B_transithedu', 0, None, None, 0)
B_rhhedu = Beta('B_rhhedu', 0, None, None, 0)
B_walkhedu = Beta('B_walkhedu', 0, None, None, 0)
B_bikehedu = Beta('B_bikehedu', 0, None, None, 0)
B_microhedu = Beta('B_microhedu', 0, None, None, 0)

B_caremp = Beta('B_caremp', 0, None, None, 0)
B_transitemp = Beta('B_transitemp', 0, None, None, 0)
B_rhemp = Beta('B_rhemp', 0, None, None, 0)
B_walkemp = Beta('B_walkemp', 0, None, None, 0)
B_bikeemp = Beta('B_bikeemp', 0, None, None, 0)
B_microemp = Beta('B_microemp', 0, None, None, 0)


B_carhhsize = Beta('B_carhhsize', 0, None, None, 0)
B_transithhsize = Beta('B_transithhsize', 0, None, None, 0)
B_rhhhsize = Beta('B_rhhhsize', 0, None, None, 0)
B_walkhhsize = Beta('B_walkhhsize', 0, None, None, 0)
B_bikehhsize = Beta('B_bikehhsize', 0, None, None, 0)
B_microhhsize = Beta('B_microhhsize', 0, None, None, 0)


B_carPBIKE = Beta('B_carPBIKE', 0, None, None, 0)
B_transitPBIKE = Beta('B_transitPBIKE', 0, None, None, 0)
B_rhPBIKE = Beta('B_rhPBIKE', 0, None, None, 0)
B_walkPBIKE = Beta('B_walkPBIKE', 0, None, None, 0)
B_bikePBIKE = Beta('B_bikePBIKE', 0, None, None, 0)
B_microPBIKE = Beta('B_microPBIKE', 0, None, None, 0)

B_carpop = Beta('B_carpop', 0, None, None, 0)
B_transitpop = Beta('B_transitpop', 0, None, None, 0)
B_rhpop = Beta('B_rhpop', 0, None, None, 0)
B_walkpop = Beta('B_walkpop', 0, None, None, 0)
B_bikepop = Beta('B_bikepop', 0, None, None, 0)
B_micropop = Beta('B_micropop', 0, None, None, 0)


B_cost = Beta('B_cost', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)


In [ ]:
# Definition of the utility functions
V1 = (ASC_CAR#_RND 
      + B_carTIME * cartime 
      + B_carhhsize*hhsize
     )
V2 = (ASC_TRANSIT#_RND 
      + B_transitTIME * transittime 
      + B_transitHBSHOP*trippurp_HBSHOP
      + B_transitHBSOC*trippurp_HBSOC
      + B_transitHBO*trippurp_HBO
      + B_transitNHB*trippurp_NHB
      )

V3 = (ASC_RH#_RND 
      + B_rhTIME * rdtime 
      + B_rhHBSHOP*trippurp_HBSHOP
      + B_rhHBSOC*trippurp_HBSOC
      + B_rhHBO*trippurp_HBO
      + B_rhNHB*trippurp_NHB
      + B_rhemp*employ
      + B_rhhhsize*hhsize
     )

V4 = (ASC_WALK#_RND 
      + B_walkTIME * walktime 
      + B_walkHBSHOP*trippurp_HBSHOP
      + B_walkHBSOC*trippurp_HBSOC
      + B_walkHBO*trippurp_HBO
      + B_walkNHB*trippurp_NHB
      )

V5 = (ASC_BIKE#_RND 
      + B_bikeTIME * biketime
      + B_bikeHBSHOP*trippurp_HBSHOP
      + B_bikeHBSOC*trippurp_HBSOC
      + B_bikeHBO*trippurp_HBO
      + B_bikeNHB*trippurp_NHB
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      + B_bikeBKLN3*(BKLN_3 + BKLN_4)
     )

#V6 micromobility
V7 = (ASC_SCOOTER#_RND 
      + B_scooterTIME*sctime 
      + B_costadj*sccost_adj
      + B_access*SCAW*(1-AVtech) + B_waitav*SCAV*AVtech + B_AV*AVtech 
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      +B_bikeBKLN3*(BKLN_3 + BKLN_4)
      + B_microHBSHOP*trippurp_HBSHOP
      + B_microHBSOC*trippurp_HBSOC
      + B_microHBO*trippurp_HBO
      + B_microNHB*trippurp_NHB
      + B_microemp*employ
      + B_microPBIKE*bike
      + B_micropop*popdensityk
      )

V8 = (ASC_DLBIKE#_RND 
      + B_bikeTIME*dltime 
      + B_costadj*dlcost_adj 
      + B_access*DLAW
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      + B_bikeBKLN3*(BKLN_3 + BKLN_4)
      + B_microHBSHOP*trippurp_HBSHOP
      + B_microHBSOC*trippurp_HBSOC
      + B_microHBO*trippurp_HBO
      + B_microNHB*trippurp_NHB
      + B_microemp*employ
      + B_microPBIKE*bike
      + B_micropop*popdensityk
    
     )

V9 = (ASC_DKBIKE#_RND 
      + B_bikeTIME*dbtime
      + B_costadj*dbcost_adj 
      + B_access*DBAW + B_drop*DBDW 
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      + B_bikeBKLN3*(BKLN_3 + BKLN_4)
      + B_microHBSHOP*trippurp_HBSHOP
      + B_microHBSOC*trippurp_HBSOC
      + B_microHBO*trippurp_HBO
      + B_microNHB*trippurp_NHB
      + B_microemp*employ
      + B_microPBIKE*bike
      + B_micropop*popdensityk

     )

V10 = (ASC_SCTRANSIT#_RND 
       + B_scooterTIME*sttime #
       + B_transitTIME*stransittime 
       + B_costadj*sttocost_adj 
       + B_access*STAW*(1-AVtech ) + B_waitav*STAV*AVtech + B_AV*AVtech  
       + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
    
       +B_bikeBKLN3*(BKLN_3 + BKLN_4)
       #+ B_TEMPsc*TEMP + B_TEMPsqsc*TEMP2
       #+ B_PURP2*purpose_2 + B_PURP3*purpose_3 + B_PURP4*purpose_4 + B_PURP5*purpose_5 + B_PURP6*purpose_6
       + B_microHBSHOP*trippurp_HBSHOP
       + B_microHBSOC*trippurp_HBSOC
       + B_microHBO*trippurp_HBO
       + B_microNHB*trippurp_NHB
       + B_microemp*employ
       + B_microPBIKE*bike
       + B_micropop*popdensityk
       
      )

